# 01 Multidimensional multi Knapsack Problem

## Problem Overview

The multidimensional multi knapsack problem (MMKP) is a generalization of the classic knapsack problem. In this problem, we have multiple knapsacks, each with its own capacity constraints across multiple dimensions (e.g., weight, volume, etc.). We also have a set of items, each with a value and a set of weights corresponding to each dimension. The goal is to maximize the total value of the items placed in the knapsacks without exceeding the capacity constraints of any knapsack in any dimension. 

### Attributes
- The `CONSTRAINTS` array define for each knapsack, what is the maximum weight and what volume it can carry.
- The `WEIGHTS` array defines for each item, what are the dimensions of the item (weight, volume, ...).
- The `VALUES` array defines for each item, what is its value.

### Example Output

with 3 kapsacks and 10 items with 2 dimensions (weight and volume):

```bash
(array([[287, 117],
        [319, 114],
        [204, 244]]),
 array([[52, 69],
        [13, 86],
        [89, 76],
        [54, 52],
        [25, 89],
        [ 5, 87],
        [50, 19],
        [97, 60],
        [85, 27],
        [97, 79]]),
 array([64, 73, 52, 71,  0, 95,  0, 34, 31, 21]))
```

> Note: It is need less to say that the items are the indecises of the arrays like 0, 1, 2, ..., n-1 in `VALUES` and `WEIGHTS`. **and one items can be in only one knapsack.**

In [6]:
import numpy as np
from tqdm import tqdm
from collections import deque

In [2]:
# Problem 1 
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

In [2]:
def value(solution : np.ndarray) -> int:
    return (VALUES * np.any(solution, axis=0)).sum()

def valid(solution : np.ndarray) -> bool:
    return np.all(WEIGHTS.T@solution.T <= CONSTRAINTS.T)

def get_items_left(solution: np.ndarray):
    '''
    Since each item can only be in one knapsack, we just check indices of current solution for all knapsacks are empty, if so, that item is left outside of knapsacks.
    '''
    items_in_knapsacks = np.any(solution, axis=0)
    return np.where(~items_in_knapsacks)[0]

In [3]:
def tweak(solution : np.ndarray) -> np.ndarray:
    '''
    tweak the solution by randomly assigning one **unassigned item** to a random knapsack.

    By checking unassigned items, we ensure that we do not put an item in two knapsacks.
    '''
    new_solution = solution.copy()
    # what are the items left to be assigned
    items_left = get_items_left(new_solution)
    if len(items_left) == 0:
        return new_solution
    # randomly pick one items in range [0, NUM_ITEMS)
    item = np.random.choice(items_left) # this is the index in KANAPSACK
    # randomly pick one knapsack in range [0, NUM_KNAPSACKS)
    knapsack = np.random.randint(0, NUM_KNAPSACKS)
    # assign the item to the knapsack
    new_solution[knapsack, item] = True
    
    return new_solution

def sa_tweak(solution : np.ndarray, strength : float = 0.5) -> np.ndarray:
    '''
    Simulated Annealing tweak
    '''
    
    new_solution = solution.copy()
    # what are the items left to be assigned
    items_left = get_items_left(new_solution)
    if len(items_left) == 0:
        return new_solution
    
    again = True
    while again:

        item = np.random.choice(items_left) # this is the index in KANAPSACK
        knapsack = np.random.randint(0, NUM_KNAPSACKS)
        new_solution[knapsack, item] = True
        again = np.random.rand() < strength

    return new_solution

In [5]:
def test_tweak(solution):
    new_solution = solution.copy()
    for _ in range(5):
        new_solution = tweak(new_solution)
        items_left = get_items_left(new_solution)

        print("Items left:", items_left, "Count:", len(items_left))
        print("Value:", value(new_solution))
        print("Valid:", valid(new_solution))
        # print(new_solution)
test_tweak(solution)

Items left: [0 1 2 3 4 5 7 8 9] Count: 9
Value: 8
Valid: False
Items left: [0 1 2 3 5 7 8 9] Count: 8
Value: 51
Valid: False
Items left: [1 2 3 5 7 8 9] Count: 7
Value: 59
Valid: False
Items left: [1 2 5 7 8 9] Count: 6
Value: 102
Valid: False
Items left: [1 2 5 7 9] Count: 5
Value: 122
Valid: False


In [6]:
from collections import deque

def hill_climbing(solution: np.ndarray, max_iter=1000, max_messages=5, print_every=100):
    best_solution = solution.copy()
    best_value = value(best_solution)
    messages = deque(maxlen=max_messages)
    
    for iteration in tqdm(range(max_iter), desc="Hill Climbing"):
        new_solution = tweak(best_solution)

        items_left = len(get_items_left(new_solution))
        
        if valid(new_solution):
            new_value = value(new_solution)
            if new_value > best_value:
                best_value = new_value
                best_solution = new_solution
                messages.append(f"Iter {iteration}: value = {best_value}, item left = {items_left}")
            if new_value == best_value:
                best_solution = new_solution

        if iteration % print_every == 0 and messages:
            tqdm.write("\n".join(messages))
            messages.clear()

        if items_left == 0:
            tqdm.write("All items assigned!")
            break

    # tqdm.write(f"Iter {iteration + 1}: Best value = {best_value}, items left = {len(get_items_left(best_solution))}")
    return best_solution, best_value, items_left

# Probem 1 Configuration

In [7]:
# Problem 1 
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = hill_climbing(solution, max_iter=10000)
print(100 * "-")
print("Best value found for problem 1:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Hill Climbing:   0%|          | 41/10000 [00:00<00:03, 2854.31it/s]

Iter 0: value = 51, item left = 19
All items assigned!
----------------------------------------------------------------------------------------------------
Best value found for problem 1: 994


In [9]:
def sa_hill_climbing(solution: np.ndarray, max_iter=1000, max_messages=5,
                    print_every=100, initial_temp=1.0, cooling_rate=0.99):
    best_solution = solution.copy()
    best_value = value(best_solution)
    temp = initial_temp
    messages = deque(maxlen=max_messages)

    for iteration in tqdm(range(max_iter), desc="Simulated Annealing Hill Climbing"):
        new_solution = sa_tweak(best_solution, strength=0.8)
        items_left = len(get_items_left(new_solution))

        if valid(new_solution):
            new_value = value(new_solution)
            if new_value > best_value:
                best_value = new_value
                best_solution = new_solution
                messages.append(f"Iter {iteration}: value = {best_value}, item left = {items_left}")
            elif new_value == best_value: # moving is better than staying ! 
                best_solution = new_solution
            else: # SA 
                # accept with a probability of exp((new_value - best_value) / temp)
                prob = np.exp((new_value - best_value) / temp)
                if np.random.rand() < prob:
                    best_solution = new_solution

        temp *= cooling_rate

        if iteration % print_every == 0 and messages:
            tqdm.write("\n".join(messages))
            messages.clear()

        if items_left == 0:
            tqdm.write("All items assigned!")
            break

    return best_solution, best_value, items_left

# Problem 2 Configuration

In [15]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = sa_hill_climbing(solution, max_iter=100000,
                                                             initial_temp=10.0, cooling_rate=0.8)
print(100 * "-")
print("Best value found for problem 2:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Simulated Annealing Hill Climbing:   2%|▏         | 1761/100000 [00:00<00:11, 8882.11it/s]

Iter 49: value = 23825, item left = 53
Iter 59: value = 23917, item left = 52
Iter 60: value = 24362, item left = 51
Iter 63: value = 24674, item left = 50
Iter 70: value = 26274, item left = 48
Iter 115: value = 27035, item left = 47
Iter 137: value = 27936, item left = 45
Iter 144: value = 28162, item left = 44
Iter 152: value = 28487, item left = 43
Iter 159: value = 28530, item left = 42
Iter 207: value = 29216, item left = 41
Iter 260: value = 30156, item left = 40
Iter 475: value = 31078, item left = 39
Iter 568: value = 31755, item left = 38
Iter 1209: value = 32391, item left = 37
Iter 1221: value = 32519, item left = 36
Iter 1368: value = 32746, item left = 35


Simulated Annealing Hill Climbing:   5%|▍         | 4730/100000 [00:00<00:09, 9738.04it/s]

Iter 3493: value = 32822, item left = 34


Simulated Annealing Hill Climbing: 100%|██████████| 100000/100000 [00:09<00:00, 10945.00it/s]

----------------------------------------------------------------------------------------------------
Best value found for problem 2: 32822
But there are still 26 items left unassigned.


# Problem 3 

In [16]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = sa_hill_climbing(solution, max_iter=1000,
                                                             initial_temp=10.0, cooling_rate=0.8)
print(100 * "-")
print("Best value found for problem 3:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Simulated Annealing Hill Climbing:   0%|          | 1/1000 [00:00<03:33,  4.68it/s]

Iter 0: value = 1217, item left = 4997


Simulated Annealing Hill Climbing:  10%|█         | 101/1000 [00:22<02:58,  5.05it/s]

Iter 96: value = 243135, item left = 4518
Iter 97: value = 246935, item left = 4511
Iter 98: value = 247423, item left = 4510
Iter 99: value = 248634, item left = 4506
Iter 100: value = 252084, item left = 4499


Simulated Annealing Hill Climbing:  20%|██        | 201/1000 [00:43<02:40,  4.98it/s]

...

Iter 753: value = 909272, item left = 3178
Iter 755: value = 909974, item left = 3177
Iter 760: value = 911588, item left = 3175
Iter 780: value = 912647, item left = 3173
Iter 794: value = 913375, item left = 3172


Simulated Annealing Hill Climbing:  90%|█████████ | 902/1000 [03:17<00:20,  4.72it/s]

Iter 876: value = 927872, item left = 3146
Iter 889: value = 928857, item left = 3145
Iter 896: value = 929970, item left = 3142
Iter 898: value = 930614, item left = 3141
Iter 899: value = 930690, item left = 3140


Simulated Annealing Hill Climbing: 100%|██████████| 1000/1000 [03:36<00:00,  4.63it/s]

----------------------------------------------------------------------------------------------------
Best value found for problem 3: 937058
But there are still 3121 items left unassigned.
